## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split
from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.laplacian_kernel = torch.tensor([[0.,  1., 0.],
                                              [1., -4., 1.],
                                              [0.,  1., 0.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)
        #laplacian_e1 = F.conv2d(first_channel, self.laplacian_kernel, padding=1)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)
        #laplacian_e2 = F.conv2d(second_channel, self.laplacian_kernel, padding=1)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        e1_weighted = first_channel / (third_channel + epsilon)
        e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            #grad_e1_x,         # Gradient X of channel1
            #grad_e1_y,         # Gradient Y of channel1
            #grad_e2_x,         # Gradient X of channel2
            #grad_e2_y,         # Gradient Y of channel2
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
            #laplacian_e1,      # Laplacian of channel1
            #laplacian_e2,      # Laplacian of channel2
            e1_weighted,       # e1_weighted
            e2_weighted        # e2_weighted
        ], dim=1)

        return output

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder()
        # Encoder
        self.encoder1 = DoubleConv(in_channels+6, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Agregar features
        x = self.channel_adder(x)
        
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Creación particion inicial

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Carga modelo

In [8]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()

In [14]:
current_partition = 1

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.53it/s, WMAPE=4.17, DICE=0.131, DPEAKS=94.6, Loss=0.013]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.55it/s, WMAPE=2.37, DICE=0.103, DPEAKS=87.9, Loss=0.00815]


Saving best model
Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:11<00:00,  6.57it/s, WMAPE=5.81, DICE=0.215, DPEAKS=144, Loss=0.022]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:10<00:00,  7.06it/s, WMAPE=3.11, DICE=0.074, DPEAKS=61.3, Loss=0.00956]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:42<00:00,  6.87it/s, WMAPE=1.91, DICE=0.0763, DPEAKS=59.1, Loss=0.00697]


In [10]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_val: 100%|██████████| 77/77 [00:11<00:00,  6.71it/s, WMAPE=1.97, DICE=0.0756, DPEAKS=58.4, Loss=0.00698]


In [15]:
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
current_partition=2

In [16]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:10<00:00,  7.16it/s, WMAPE=18.2, DICE=0.365, DPEAKS=183, Loss=0.0738]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.45it/s, WMAPE=3.16, DICE=0.0997, DPEAKS=87, Loss=0.00904]


Saving best model
Epoch [3/50]


...

Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:10<00:00,  7.63it/s, WMAPE=3.65, DICE=0.0875, DPEAKS=78.7, Loss=0.0102]


Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:10<00:00,  7.61it/s, WMAPE=4.84, DICE=0.0787, DPEAKS=76.3, Loss=0.0125]


early stopping: 3 epochs without improvement
Training complete.


In [17]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_3_val: 100%|██████████| 77/77 [00:11<00:00,  6.84it/s, WMAPE=2.93, DICE=0.0991, DPEAKS=74.3, Loss=0.0087]


In [18]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [19]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  8.18it/s, WMAPE=2.41, DICE=0.0843, DPEAKS=82.4, Loss=0.00723]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.11it/s, WMAPE=2.17, DICE=0.0709, DPEAKS=68.3, Loss=0.00632]


Saving best model
Epoch [3/50]


...

Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:09<00:00,  7.73it/s, WMAPE=3.06, DICE=0.118, DPEAKS=97.3, Loss=0.00865]


Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:09<00:00,  7.72it/s, WMAPE=15.7, DICE=0.233, DPEAKS=130, Loss=0.0429]


early stopping: 3 epochs without improvement
Training complete.


In [20]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_4_val: 100%|██████████| 77/77 [00:10<00:00,  7.34it/s, WMAPE=1.96, DICE=0.0701, DPEAKS=58, Loss=0.00606]


In [21]:
current_partition=4

In [22]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:10<00:00,  7.70it/s, WMAPE=1.97, DICE=0.0474, DPEAKS=47.5, Loss=0.00528]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.07it/s, WMAPE=1.91, DICE=0.0468, DPEAKS=46, Loss=0.00514]


Saving best model
Epoch [3/50]


...

Epoch [16/50]


Validate Epoch 16: 100%|██████████| 77/77 [00:09<00:00,  8.04it/s, WMAPE=1.84, DICE=0.0444, DPEAKS=37.6, Loss=0.00502]


Epoch [17/50]


Validate Epoch 17: 100%|██████████| 77/77 [00:09<00:00,  7.77it/s, WMAPE=1.92, DICE=0.0439, DPEAKS=38.2, Loss=0.00525]


early stopping: 3 epochs without improvement
Training complete.


In [23]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_5_val: 100%|██████████| 77/77 [00:10<00:00,  7.21it/s, WMAPE=1.66, DICE=0.0433, DPEAKS=33.3, Loss=0.0047]
